In [1]:
import pandas as pd
import numpy as np

In [4]:
# loading imdb actor information
actors_raw = pd.read_csv('data/name.basics.tsv', sep='\t')

# filter out rows with missing data cruical for analysis
actors_raw = actors_raw[actors_raw["birthYear"] != r"\N"]

# loading imdb's principals
principals_raw = pd.read_csv('data/title.principals.tsv', sep='\t')

In [5]:
print("Number of actors:", actors_raw.size)
actors_raw

Number of actors: 3411282


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0050419,tt0053137,tt0072308,tt0045537"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0038355,tt0037382,tt0117057,tt0071877"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0054452,tt0056404,tt0057345,tt0049189"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0080455,tt0077975,tt0078723,tt0072562"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0060827,tt0083922,tt0050976,tt0050986"
...,...,...,...,...,...,...
12148488,nm9993436,Frank J. Gaily,1915,2008,sound_department,tt0189339
12148570,nm9993526,Ben Ray Lujan,1972,\N,NaN,"tt0476038,tt7516996,tt15385660,tt4209386"
12148578,nm9993535,Henry Lawfull,2006,\N,actor,"tt10187208,tt5900600"
12148699,nm9993675,Ebrahim Alkazi,1925,2020,NaN,\N


In [6]:
#print("Number of movies considered in 'principals' data set", len(list(dict.fromkeys(principals["tconst"]))))
principals_raw

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
53573728,tt9916880,4,nm10535738,actress,\N,"[""Horrid Henry""]"
53573729,tt9916880,5,nm0996406,director,principal director,\N
53573730,tt9916880,6,nm1482639,writer,\N,\N
53573731,tt9916880,7,nm2586970,writer,books,\N


## Filter for leading actors and add column for gender
Filter for leading actors based on imdb's principals data which contains for each movie the most important people (i. e. actors, actress, directors etc.). The importance is given by imdb and stored in the 'ordering' column

In [7]:
# filter for actors and actress in principals
principals = principals_raw[(principals_raw["category"] == "actress") | (principals_raw["category"] == "actor")] 

# add gender column (0=male, 1=female)
principals["gender"] = 0
principals.loc[principals["category"] == "actress"] = 1

# select ony important columns of principal and actors data set and join them left
principal_actors = principals[["tconst", "ordering", "nconst", "gender"]].merge(actors_raw[["nconst",	"primaryName",	"birthYear", "deathYear"]], how="left", on="nconst")
print("Number of male actors:", sum(principal_actors["gender"]))
print("Number of actors:", principal_actors.size)

principal_actors

/var/folders/dk/kt0tgs8j1cb1k_hn1_h2bch80000gn/T/ipykernel_72105/2856137759.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  principals["gender"] = 0


Number of male actors: 9152278
Number of actors: 146907740


,tconst,ordering,nconst,gender,primaryName,birthYear,deathYear
0,tt0000005,1,nm0443482,0,Charles Kayser,1878,1966
1,tt0000005,2,nm0653042,0,John Ott,1850,1931
2,tt0000007,1,nm0179163,0,James J. Corbett,1866,1933
3,tt0000007,2,nm0183947,0,Peter Courtney,1867,1896
4,tt0000008,1,nm0653028,0,Fred Ott,1860,1936
...,...,...,...,...,...,...,...
20986815,1,1,1,1,NaN,NaN,NaN
20986816,tt9916880,1,nm1483166,0,NaN,NaN,NaN
20986817,1,1,1,1,NaN,NaN,NaN
20986818,tt9916880,3,nm0286175,0,NaN,NaN,NaN


In [52]:
# Find total number of movies played by every leading actor
unique_actors = principal_actors["nconst"].unique()
print("There's a total of", len(unique_actors) - 1, "unique actors.")

There's a total of 1435557 unique actors.


In [49]:
total_actor_experience = principal_actors.groupby(["nconst"]).count()
total_actor_experience

,tconst,ordering,gender,primaryName,birthYear,deathYear
nconst,,,,,,
1,9152278,9152278,9152278,0,0,0
nm0000001,56,56,56,56,56,56
nm0000004,50,50,50,50,50,50
nm0000005,6,6,6,6,6,6
nm0000007,75,75,75,75,75,75
...,...,...,...,...,...,...
nm9993694,4,4,4,0,0,0
nm9993697,1,1,1,0,0,0
nm9993698,1,1,1,0,0,0


In [50]:
actor_to_experience = {}
for row in total_actor_experience.iterrows():
    actor_to_experience[row[0]] = row[1]["tconst"]
del actor_to_experience[1] #remove the useless total count value there was

## Add column for age of actor during start of movie

In [9]:
# load in movie data and select relevent columns: tconst and startYear
movies_raw = pd.read_csv("data/title.basics.tsv", sep="\t")[["tconst", "startYear", "genres"]]

# filter out rows with missing data cruical for analysis
movies_raw = movies_raw[movies_raw["startYear"] != r"\N"]

/var/folders/dk/kt0tgs8j1cb1k_hn1_h2bch80000gn/T/ipykernel_72105/2564971414.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_raw = pd.read_csv("data/title.basics.tsv", sep="\t")[["tconst", "startYear", "genres"]]


In [10]:
# join movie data with actors
actor_movie_combi = principal_actors.merge(movies_raw, how="left", on="tconst")

# cast year columns to numeric in order to calculate
actor_movie_combi["startYear"] = pd.to_numeric(actor_movie_combi["startYear"]) # errors parameter for the case when date not given
actor_movie_combi["birthYear"] = pd.to_numeric(actor_movie_combi["birthYear"]) # errors parameter for the case when date not given
actor_movie_combi["deatYear"] = pd.to_numeric(actor_movie_combi["deathYear"], errors="coerce") # errors parameter for the case when actor is not dead, then NaN

# add age_at_movie_start column
actor_movie_combi["age_at_movie_start"] = actor_movie_combi["startYear"] - actor_movie_combi["birthYear"]
print("Number of movie actor combinations:", actor_movie_combi.size)

actor_movie_combi

Number of movie actor combinations: 230855020


,tconst,ordering,nconst,gender,primaryName,birthYear,deathYear,startYear,genres,deatYear,age_at_movie_start
0,tt0000005,1,nm0443482,0,Charles Kayser,1878.0,1966,1893.0,"Comedy,Short",1966.0,15.0
1,tt0000005,2,nm0653042,0,John Ott,1850.0,1931,1893.0,"Comedy,Short",1931.0,43.0
2,tt0000007,1,nm0179163,0,James J. Corbett,1866.0,1933,1894.0,"Short,Sport",1933.0,28.0
3,tt0000007,2,nm0183947,0,Peter Courtney,1867.0,1896,1894.0,"Short,Sport",1896.0,27.0
4,tt0000008,1,nm0653028,0,Fred Ott,1860.0,1936,1894.0,"Documentary,Short",1936.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...
20986815,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20986816,tt9916880,1,nm1483166,0,NaN,NaN,NaN,2014.0,"Adventure,Animation,Comedy",NaN,NaN
20986817,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20986818,tt9916880,3,nm0286175,0,NaN,NaN,NaN,2014.0,"Adventure,Animation,Comedy",NaN,NaN


## Add column for imdb ratings

In [33]:
# load in imdb ratings
ratings_raw = pd.read_csv('data/title.ratings.tsv', sep='\t')

In [34]:
# add ratings to overall data set
actor_movie_combi = actor_movie_combi.merge(ratings_raw, how="left", on="tconst")

actor_movie_combi

,tconst,ordering,nconst,gender,primaryName,birthYear,deathYear,startYear,genres,deatYear,age_at_movie_start,averageRating,numVotes
0,tt0000005,1,nm0443482,0,Charles Kayser,1878.0,1966,1893.0,"Comedy,Short",1966.0,15.0,6.2,2554.0
1,tt0000005,2,nm0653042,0,John Ott,1850.0,1931,1893.0,"Comedy,Short",1931.0,43.0,6.2,2554.0
2,tt0000007,1,nm0179163,0,James J. Corbett,1866.0,1933,1894.0,"Short,Sport",1933.0,28.0,5.4,797.0
3,tt0000007,2,nm0183947,0,Peter Courtney,1867.0,1896,1894.0,"Short,Sport",1896.0,27.0,5.4,797.0
4,tt0000008,1,nm0653028,0,Fred Ott,1860.0,1936,1894.0,"Documentary,Short",1936.0,34.0,5.4,2069.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20986815,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20986816,tt9916880,1,nm1483166,0,NaN,NaN,NaN,2014.0,"Adventure,Animation,Comedy",NaN,NaN,NaN,NaN
20986817,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20986818,tt9916880,3,nm0286175,0,NaN,NaN,NaN,2014.0,"Adventure,Animation,Comedy",NaN,NaN,NaN,NaN


## Add column for number of movies an actor has been in before start of respective movie

In [80]:
actor_movie_combi["exp_so_far"] = 0
from collections import defaultdict
exp_so_far = defaultdict(int)
for row in actor_movie_combi.iterrows():
    actor = row[1]["nconst"]
    actor_movie_combi.loc[row[0], "exp_so_far"] = exp_so_far[actor]
    exp_so_far[actor] += 1

# Drop rows with NA values and save to CSV
actor_movie_combi.dropna(inplace=True) 
actor_movie_combi.to_csv("data/actor_movie_combi.csv")

In [10]:
actor_movie_combi

,tconst,ordering,nconst,gender,primaryName,birthYear,deathYear,startYear,genres,deatYear,age_at_movie_start,averageRating,numVotes,exp_so_far
0,tt0000005,1,nm0443482,0,Charles Kayser,1878.0,1966,1893.0,"Comedy,Short",1966.0,15.0,6.2,2554.0,0
1,tt0000005,2,nm0653042,0,John Ott,1850.0,1931,1893.0,"Comedy,Short",1931.0,43.0,6.2,2554.0,0
2,tt0000007,1,nm0179163,0,James J. Corbett,1866.0,1933,1894.0,"Short,Sport",1933.0,28.0,5.4,797.0,0
3,tt0000007,2,nm0183947,0,Peter Courtney,1867.0,1896,1894.0,"Short,Sport",1896.0,27.0,5.4,797.0,0
4,tt0000008,1,nm0653028,0,Fred Ott,1860.0,1936,1894.0,"Documentary,Short",1936.0,34.0,5.4,2069.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20979631,tt9909866,3,nm0619077,0,Ichirô Nagai,1931.0,2014,1985.0,"Action,Adventure,Animation",2014.0,54.0,4.6,20.0,2728
20986391,tt9916406,1,nm0682935,0,Ryszard Pietruski,1922.0,1996,1976.0,Crime,1996.0,54.0,5.8,5.0,34
20986392,tt9916406,2,nm0468565,0,August Kowalczyk,1921.0,2012,1976.0,Crime,2012.0,55.0,5.8,5.0,23
20986393,tt9916406,3,nm0387710,0,Zygmunt Hobot,1930.0,2004,1976.0,Crime,2004.0,46.0,5.8,5.0,3


In [9]:
actor_movie_combi = pd.read_csv("data/actor_movie_combi.csv")

## Add column for number of movies per genre an actor has been in before start of respective movie

In [ ]:
# TODO
# NOTE wait for arvind's genre filtering
# make for each genre a column which contains the number of movies per said genre an actor has been in before start of respective movie

## Cleaning data